# Initialization 

In [ ]:
import numpy as np
from PIL import Image
import os
import requests
from io import BytesIO

import tensorflow as tf
slim = tf.contrib.slim

In [ ]:
IMAGE_SIZE = 299

TENSORFLOW_MODELS_ROOT = '.'
INCEPTION_CHECKPOINT_DIR = '.'
CHECKPOINT_FILE = os.path.join(
    INCEPTION_CHECKPOINT_DIR, 'inception_resnet_v2_2016_08_30.ckpt')

MALAMUTE_PATH = './images/malamute.png'
HUSKY_PATH = './images/Siberian-husky.jpg'

MALAMUTE_IMAGE = Image.open(MALAMUTE_PATH)
HUSKY_IMAGE = Image.open(HUSKY_PATH)

In [ ]:
def convertToJpeg(im):
    with BytesIO() as f:
        im.save(f, format='JPEG')
        return f.getvalue()
    

def create_readable_names_for_imagenet_labels():
    """Create a dict mapping label id to human readable string.
    Returns:
      labels_to_names: dictionary where keys are integers from to 1000
      and values are human-readable names.
    We retrieve a synset file, which contains a list of valid synset labels used
    by ILSVRC competition. There is one synset one per line, eg.
          #   n01440764
          #   n01443537
    We also retrieve a synset_to_human_file, which contains a mapping from synsets
    to human-readable names for every synset in Imagenet. These are stored in a
    tsv format, as follows:
          #   n02119247    black fox
          #   n02119359    silver fox
    We assign each synset (in alphabetical order) an integer, starting from 1
    (since 0 is reserved for the background class).
    """

    # pylint: disable=g-line-too-long
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/slim/datasets'
    synset_url = '{}/imagenet_lsvrc_2015_synsets.txt'.format(base_url)
    synset_to_human_url = '{}/imagenet_metadata.txt'.format(base_url)

    synset_list = [s.strip() for s in requests.get(synset_url).content.decode().split('\n') if len(s.strip())]
    num_synsets_in_ilsvrc = len(synset_list)
    print(synset_list[-1])
    assert num_synsets_in_ilsvrc == 1000

    synset_to_human_list = requests.get(synset_to_human_url).content.decode().split('\n')[:-1]
    num_synsets_in_all_imagenet = len(synset_to_human_list)
    assert num_synsets_in_all_imagenet == 21842

    synset_to_human = {}
    for s in synset_to_human_list:
        parts = s.strip().split('\t')
        assert len(parts) == 2
        synset = parts[0]
        human = parts[1]
        synset_to_human[synset] = human

    label_index = 1
    labels_to_names = {0: 'background'}
    for synset in synset_list:
        name = synset_to_human[synset]
        labels_to_names[label_index] = name
        label_index += 1

    return labels_to_names

labels_mapping = create_readable_names_for_imagenet_labels()

In [ ]:
from scipy.misc import imresize
from skimage.io import imread, imsave, imshow

%matplotlib inline  
import matplotlib.pyplot as plt

# Increase plot sizes
plt.rcParams['figure.figsize'] = (15, 9)

In [ ]:
import sys
sys.path.append(os.path.join(
    TENSORFLOW_MODELS_ROOT, 'models/research/slim/nets'))
from inception_resnet_v2 import *
import numpy as np

In [ ]:
input_tensor = tf.placeholder(tf.float32, shape=(None,IMAGE_SIZE,IMAGE_SIZE,3), name='input_image')
scaled_input_tensor = tf.scalar_mul((1.0/255), input_tensor)
scaled_input_tensor = tf.subtract(scaled_input_tensor, 0.5)
scaled_input_tensor = tf.multiply(scaled_input_tensor, 2.0)

print(scaled_input_tensor)
#Load the model
arg_scope = inception_resnet_v2_arg_scope()
with slim.arg_scope(arg_scope):
    logits, end_points = inception_resnet_v2(scaled_input_tensor, is_training=False, reuse=None)

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, CHECKPOINT_FILE)

print('====== ENDPOINTS ======')
print(end_points.keys())

# Prediction

In [ ]:
def predict_image(image, target_tensors):
    im = image.convert('RGB').resize((IMAGE_SIZE,IMAGE_SIZE))
    np_im = np.array(im)
    np_im = np_im.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
    return sess.run(target_tensors, feed_dict={input_tensor: np_im}) 

sample_images = [MALAMUTE_IMAGE, HUSKY_IMAGE]
for image in sample_images:
    predict_values, logit_values = predict_image(image, [end_points['Predictions'], logits])
    print (np.max(predict_values), np.max(logit_values))
    predicted_label = np.argmax(predict_values)
    print ("{} : {}".format(predicted_label, labels_mapping[predicted_label]))

# Helper functions

In [ ]:
from PIL import ImageDraw 


def add_black_patch(image, coordinates):
    res_image = image.copy()
    draw = ImageDraw.Draw(res_image)
    black = '#000000'
    draw.rectangle(coordinates, fill=black, outline=black)
    return res_image


def overlay(image, heat_map):
    t = np.pad(heat_map, (10, 10), mode='constant')
    t = t[10:, 10:]
    res = np.zeros(shape=(IMAGE_SIZE, IMAGE_SIZE, 4))
    t2 = (t > 0) * t
    t1 = (t < 0) * -t
    
    minv, maxv = min(t1.min(), t2.min()), max(t1.max(), t2.max())
    scale_factor = 1.0
    if maxv != minv:
        scale_factor = maxv - minv
    res[:, :, 0] = (t2 - minv) / scale_factor
    res[:, :, 2] = (t1 - minv) / scale_factor
    res[:, :, 3] = res[:, :, 0] + res[:, :, 2]
    plt.imshow(image.resize((IMAGE_SIZE, IMAGE_SIZE)))
    plt.imshow(res)

# Plot specific filter position

In [ ]:
def get_heat_map(image, layer, filter_index, patch_size=10):
    resized_im = image.convert('RGB').resize((IMAGE_SIZE,IMAGE_SIZE))
    original_activations = predict_image(resized_im, [end_points[layer]]) 
    
    h, w = resized_im.size
    layer_shape = tuple(map(int,  end_points[layer].get_shape()[1:-1]))
    activation_map = np.zeros((h - patch_size, w-patch_size) + layer_shape)
    print('Desired shape' , activation_map.shape)
       
    step = 3
    for x in range(0, h - patch_size, step):
        print('Processing', x // step + 1, 'of', (h - patch_size) // step + 1)
        for y in range(0, w - patch_size, step):
            temp_im = add_black_patch(resized_im, (x, y, x + patch_size, y + patch_size))
            activations = predict_image(temp_im, [end_points[layer]])
            activation_map[y:y+step, x:x+step] = (original_activations[0] - activations[0])[0][:, :, filter_index]
    return activation_map

In [ ]:
LAYER_NAME= 'Mixed_6a'
FILTER_INDEX = 2
filter_positions = get_heat_map(MALAMUTE_IMAGE, LAYER_NAME, FILTER_INDEX)

In [ ]:
def plot_neuron2(image, filter_activations, x, y):
    selected = filter_activations[:,:, x, y]
    if selected.max() - selected.min() == 0:
        return
    
    plt.figure()
    plt.suptitle('For ({}, {})'.format(x, y))
    overlay(image, selected)   
    
for x in range(0, 8):
    for y in range(0, 8):
        plot_neuron2(MALAMUTE_IMAGE, filter_positions, x, y)

# Plot whole filter

In [ ]:
def get_mean_activation(activations):
    mean = np.mean(activations[0][0], axis=(0, 1))
    return mean

def get_filter_heat_map(image, layer, patch_size=10):
    resized_im = image.convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
    original_activations = predict_image(resized_im, [end_points[layer]])
    number_of_filters = original_activations[0].shape[-1]

    original = get_mean_activation(original_activations)
    h, w = resized_im.size
    activation_map = np.zeros((h - patch_size, w - patch_size, number_of_filters))
    print('Desired shape', activation_map.shape)
    
    step = 3
    for x in range(0, h - patch_size, step):
        print('Processing', x // step + 1, 'of', (h - patch_size) // step + 1)
        for y in range(0, w - patch_size, step):
            temp_im = add_black_patch(resized_im, (x, y, x + patch_size, y + patch_size))
            activations = predict_image(temp_im, [end_points[layer]])
            value = (original - get_mean_activation(activations))
            activation_map[y:y+step, x:x+step] = value
    return activation_map

In [ ]:
LAYER_NAME= 'Mixed_6a'
whole_filters = get_filter_heat_map(MALAMUTE_IMAGE, LAYER_NAME)

In [ ]:
for i in range(130):
    temp = whole_filters[:, :, i]
    if temp.max() - temp.min() == 0:
        continue
    plt.figure()
    plt.suptitle('For {}'.format(i))
    overlay(MALAMUTE_IMAGE, temp)

In [ ]:
def overlay_activations(image, layer, filter_index):
    resized_im = image.convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
    original_activations = predict_image(resized_im, [end_points[layer]])[0][0, :, :, filter_index]
    scaled_activations = imresize(
        original_activations, size=(IMAGE_SIZE, IMAGE_SIZE), interp='nearest', mode='L')
    scaled_activations = scaled_activations / 255
    overlay = np.zeros(shape=(IMAGE_SIZE, IMAGE_SIZE, 4))
    overlay[:, :, 0] = scaled_activations
    overlay[:, :, 1] = scaled_activations * 0.8
    overlay[:, :, 3] = scaled_activations
    plt.imshow(resized_im)
    plt.imshow(overlay)

In [ ]:
overlay_activations(MALAMUTE_IMAGE, 'Mixed_6a', 60)

# Adversarial examples

In [ ]:
def logit_with_value(tensor, coordinates, value):
    z = tf.zeros_like(tensor[0])
    indices = [coordinates]
    values = [value]
    shape = tf.shape(tensor, out_type=tf.int64)
    delta = tf.SparseTensor(indices, values, shape)
    return z + tf.sparse_tensor_to_dense(delta)

def show_image_safe(im):
    temp = im.copy()
    temp[temp > 1] = 1
    temp[temp < - 1] = -1
    imshow(temp)

def _prediction_info(predict_values, target_class_index):
    predicted_label = np.argmax(predict_values)
    class_str = "{} ({} prob: {:6f})".format(
        predicted_label, labels_mapping[predicted_label], predict_values[0][predicted_label])
    target_str = "For target class {} : {:6f}".format(
        target_class_index, predict_values[0][target_class_index])
    return "{}. {}".format(class_str, target_str)

def get_adversarial_example(image, class_index, iterations=50):
    im = image.convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
    im = np.array(im)
    im = im.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

    grad_values = logit_with_value(logits, [0, class_index], 1.0)
    grad = tf.gradients(logits, input_tensor, grad_ys=grad_values)

    res = input_tensor + grad

    modified_input = im
    for i in range(iterations):
        predict_values, modified_image = sess.run(
            [end_points['Predictions'], res], feed_dict={input_tensor: modified_input})

        print("{}: {}".format(i, _prediction_info(predict_values, class_index)))

        modified_input = modified_image[0].astype(np.uint8)


    predict_values = sess.run(end_points['Predictions'], feed_dict={input_tensor: modified_input})

    print(_prediction_info(predict_values, class_index))

    return modified_input[0]

In [ ]:
adversarial_example = get_adversarial_example(
    MALAMUTE_IMAGE, class_index=131, iterations=20)
imshow(adversarial_example)

# Deep Dream

In [ ]:
def deep_dream(image, layer_name, iterations =50):    
    im = image.convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
    im = np.array(im)
    im = im.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
    
    grad = tf.gradients(end_points[layer_name], input_tensor, grad_ys=end_points[layer_name])
    
    res = input_tensor + grad
    
    modified_input = im
    for i in range(iterations):
        modified_image = sess.run(res, feed_dict={input_tensor: modified_input})
        modified_input = modified_image[0].astype(np.uint8)
   
    
    return modified_input[0]

In [ ]:
imshow(deep_dream(MALAMUTE_IMAGE, 'Mixed_6a', iterations=100))

In [ ]:
imshow(deep_dream(MALAMUTE_IMAGE, 'Mixed_7a', iterations=100))

In [ ]:
imshow(deep_dream(MALAMUTE_IMAGE, 'Conv2d_7b_1x1', iterations=100))